<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [49]:
import re
import pandas as pd 
import numpy as np
from pprint import pprint

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')
import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary


from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [24]:
# cargue datos
data = pd.read_csv("reviews_vidjew_es.csv", sep=",", encoding="UTF-8")
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


In [45]:
stopwords_sp = stopwords.words('spanish')

In [64]:
    def pp(texto):
        texto = texto.lower()
        texto = re.sub(r"[\W\d]+", " ", texto)
        texto = texto.split() # tokenización 
        texto = [palabra for palabra in texto if palabra not in stopwords_sp]
        return texto 

    data['pp'] = data['review_body'].apply(lambda val: pp(val))
    data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [67]:
# Crear una represantación de los documentos en forma de diccionario
dictionary = Dictionary(data['pp'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['pp'].values]

In [78]:
# Entrenar el modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 5, passes=10)
model.print_topics(num_words=10)

[(0,
  '0.041*"calidad" + 0.039*"bonito" + 0.019*"cadena" + 0.017*"bien" + 0.017*"precio" + 0.014*"colgante" + 0.014*"perfecto" + 0.013*"queda" + 0.012*"mas" + 0.012*"ve"'),
 (1,
  '0.029*"calidad" + 0.023*"pulsera" + 0.021*"día" + 0.019*"esperaba" + 0.016*"llegado" + 0.015*"bonita" + 0.015*"regalo" + 0.015*"parece" + 0.014*"foto" + 0.013*"bastante"'),
 (2,
  '0.056*"juego" + 0.020*"plata" + 0.019*"buena" + 0.018*"si" + 0.016*"entrega" + 0.015*"parte" + 0.014*"amazon" + 0.013*"producto" + 0.012*"bien" + 0.012*"llegado"'),
 (3,
  '0.068*"bien" + 0.025*"si" + 0.019*"juego" + 0.014*"mejor" + 0.013*"precio" + 0.013*"así" + 0.011*"jugar" + 0.011*"dos" + 0.010*"bonitos" + 0.010*"solo"'),
 (4,
  '0.035*"producto" + 0.028*"precio" + 0.020*"buen" + 0.018*"calidad" + 0.016*"si" + 0.015*"puede" + 0.015*"caja" + 0.015*"perfecto" + 0.013*"llegó" + 0.012*"mando"')]

###  `[25 pts]` Punto 3: Visualización de LDA

In [79]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

Los textos pueden tener diferentes resultados y usos a partir de lo que se quiere analizar y de la estructuración de las bases. 
Para los modelos de clasificación se requiere modelar bajo una etiqueta y se puede predecir a partir del comportamiento de esta un resultado que ya se conoce.
Para los modelos de agrupación es indispensable que se utilicen bajo textos de un solo tema, estos permiten identificar la concentracion de grupos de registros similares y dar un panorama interesante del texto.
El LDA pueden ejecutarse bajo varios temas, este modelo permite identificar similitudes entre los textos y poder determinar un sentido en ellos.
Creo que la eficiencia de los modelos depende de la oportunidad de los mismos, considero que de acuerdo a la base que se requiera analizar y a la idea de negocio se debe deterimar el uso de uno o de otro.